In [9]:
import pandas as pd
import numpy as np

df=pd.read_csv('/content/KDD_Train.csv')
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
125969,8,udp,private,SF,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly


In [10]:
df.replace(('normal','anomaly'), (0,1), inplace=True)

In [12]:
print(pd.get_dummies(df['protocol_type']))

         icmp    tcp    udp
0       False   True  False
1       False  False   True
2       False   True  False
3       False   True  False
4       False   True  False
...       ...    ...    ...
125968  False   True  False
125969  False  False   True
125970  False   True  False
125971  False   True  False
125972  False   True  False

[125973 rows x 3 columns]


In [13]:
def dummy_df(df):
    todummy_list = ['protocol_type', 'service','flag']
    for x in todummy_list:
        dummies=pd.get_dummies(df[x],dummy_na=False)
        df = df.drop(x,axis= 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [14]:
df = dummy_df(df)
df.head(5)

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF,SH
0,0,491,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
1,0,146,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,False,False,False,False
3,0,232,8153,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False
4,0,199,420,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,True,False


In [15]:
df.isnull().sum().sort_values(ascending=False)

duration                       0
red_i                          0
printer                        0
pop_3                          0
pop_2                          0
                              ..
dst_host_rerror_rate           0
dst_host_srv_serror_rate       0
dst_host_serror_rate           0
dst_host_srv_diff_host_rate    0
SH                             0
Length: 123, dtype: int64

In [16]:
import numpy as np
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy='median')
imr.fit(df)
df = pd.DataFrame(data=imr.transform(df), columns=df.columns)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

duration                       0
red_i                          0
printer                        0
pop_3                          0
pop_2                          0
                              ..
dst_host_rerror_rate           0
dst_host_srv_serror_rate       0
dst_host_serror_rate           0
dst_host_srv_diff_host_rate    0
SH                             0
Length: 123, dtype: int64

In [20]:
x = df.drop ('class',axis= 1)
y = df['class']

In [21]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1, stratify=y)

In [23]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(x_train)
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)
print('Labels count in y:', np.bincount(y))
print('Labels count in y_train:', np.bincount(y_train))
print('Labels count in y_test:', np.bincount(y_test))

Labels count in y: [67343 58630]
Labels count in y_train: [47140 41041]
Labels count in y_test: [20203 17589]


In [25]:
from sklearn.linear_model import Perceptron

ppn = Perceptron(eta0=0.1, random_state=1)
ppn.fit(x_train_std, y_train)

Perceptron(eta0=0.1, random_state=1)

In [26]:
y_pred = ppn.predict(x_test_std)
print('Misclassified examples: %d' % (y_test != y_pred).sum())

Misclassified examples: 1209


In [27]:
from sklearn.metrics import accuracy_score

print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Accuracy: 0.968


In [28]:
print('Accuracy: %.3f' % ppn.score(x_test_std, y_test))

Accuracy: 0.968
